Importing required libraries

In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import copy
import os
from skimage.feature import blob_doh
import shutil
from csv import writer, reader
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from numpy import asarray, save, load

Uploading all data

In [ ]:
SOURCES_NUMBER = 1  # Should be equal for all data used
kSize = 5  # Size of smoothing kernel
kSigma = 1.5  # Sigma for smoothing
d = int((kSize-1)/2)
src_folder = "data/skymaps/NONE/"+str(SOURCES_NUMBER)
matrices = []
blob_pos = dict()
cont = 0
filename_dict = dict()
with open("data/xml_files/"+str(SOURCES_NUMBER)+"/blobs.csv") as csv_file:
    csv_reader = reader(csv_file, delimiter=',')
    tmp_rows = [row for row in csv_reader]
for filename in os.listdir(src_folder):
    if "NONE" in filename:
        filename_dict[int(filename.split("_")[2][:-5])] = filename
for order_id in sorted(filename_dict):
    blob_pos[cont] = []
    filename = filename_dict[order_id]
    matrices.append(fits.open(src_folder+"/"+filename))
    wcs = WCS(header=(matrices[-1])[0].header)
    for i in range(SOURCES_NUMBER):
        sky = SkyCoord(float(tmp_rows[cont][2*i+1]), float(tmp_rows[cont][2*i+2]), unit='deg')
        y, x= wcs.world_to_array_index(sky)
        blob_pos[cont].append([int(x), int(y)])
    cont += 1

Look at images

In [ ]:
for i, mat in enumerate(matrices[:5]):
    plt.matshow(mat[0].data, cmap='gray')

Defining methods required to smooth the images

In [ ]:
def gaussianKernel(size, sigma):
    kernel = np.fromfunction(lambda x, y: (1/(2*np.pi*sigma**2)) * np.e ** ((-1*((x-(size-1)/2)**2+(y-(size-1)/2)**2))/(2*sigma**2)), (size, size))
    return kernel

In [ ]:
def gaussianBlur(img, kernel):
    gaussian = np.zeros((img.shape[0]-2*d, img.shape[1]-2*d))
    for y in range(d, img.shape[0]-d):
        for x in range(d, img.shape[1]-d):
            gaussian[y-d][x-d] = np.sum(np.multiply(img[y-d:y+d+1, x-d:x+d+1], kernel))
    return gaussian

Load numpy array from npy file

In [ ]:
matrices_smoothed = load("data/skymaps/NONE/"+str(SOURCES_NUMBER)+"/matrices_smoothed.npy")
dimx = (matrices[-1][0].data).shape[0]-2*d
dimy = (matrices[-1][0].data).shape[1]-2*d
del matrices

Smoothing and saving in each image in list "matrices_smoothed".

In [ ]:
kernel = gaussianKernel(kSize, kSigma)
matrices_smoothed = []
for i, mat in enumerate(matrices):
    matrices_smoothed.append(gaussianBlur(mat[0].data, kernel))
dimx = (matrices[-1][0].data).shape[0]-2*d
dimy = (matrices[-1][0].data).shape[1]-2*d
del matrices

Save numpy array as npy file

In [ ]:
save("data/skymaps/NONE/"+str(SOURCES_NUMBER)+"/matrices_smoothed.npy", asarray(matrices_smoothed))

Showing smoothed images.

In [ ]:
print("Shape of smoothed image: ", matrices_smoothed[0].shape)

In [ ]:
for mat in matrices_smoothed[:5]:
    plt.matshow(mat, cmap='gray')

Plotting histogram of intensities

Computing mean background level

In [ ]:
mean_back = 0
for mat in matrices_smoothed:
    mean_back += mat.mean()
mean_back /= len(matrices_smoothed)
thresh = 7 * mean_back
print("Mean background level: ", mean_back)
print("Thresholding at: ", thresh)

Binarizing images using previously defined threshold.

In [ ]:
matrices_bin = []
for i, mat in enumerate(matrices_smoothed):
    matrices_bin.append(np.zeros((dimx, dimy)))
    matrices_bin[i][mat > thresh] = 1
del matrices_smoothed

In [ ]:
for mat in matrices_bin[:5]:
    plt.matshow(mat, cmap='gray')

In [ ]:
blobs = dict()
mean_error_pix = []
mean_error_angle = []
data_taken = 0
for i, mat in enumerate(matrices_bin):
    if i > 1000:
        break
    blobs[i] = []
    cont = 0
    blobs_doh = blob_doh(mat, max_sigma=20, threshold=.01)
    for blob in blobs_doh:
        y, x, r = blob
        if r > 1:
            cont += 1
            blobs[i].append((int(x), int(y)))
    if cont == SOURCES_NUMBER:
        data_taken += 1
    else:
        blobs[i] = []
print("Percentage of images with correct amount of blobs: ", data_taken/len(matrices_bin)*100, "%")
print("Showing data:")
with open("data/dataset_none/blobs.csv", "a+", newline='') as write_obj:
    csv_writer = writer(write_obj)
    for key in blobs.keys():
        blobs_key = blobs[key]
        if len(blobs_key) == SOURCES_NUMBER:
            to_be_saved = [key, SOURCES_NUMBER]
            for blob in blobs_key:
                to_be_saved.append(blob)
                closer_blob_dist = 10000
                sky_obs = wcs.pixel_to_world(blob[0], blob[1])
                w_obs = blob[0]*2*np.pi
                for i, blob_true in enumerate(blob_pos[key]):
                    sky_true = SkyCoord(float(tmp_rows[key][2*i+1]), float(tmp_rows[key][2*i+2]), unit='deg')
                    tmp_dist = (sky_true.separation(sky_obs)).degree
                    if tmp_dist < closer_blob_dist:
                        closer_blob_dist = tmp_dist
                        closer_blob = blob_true
                mean_error_angle.append(closer_blob_dist)
                mean_error_pix.append(np.sqrt((closer_blob[0]-blob[0])**2 + (closer_blob[1]-blob[1])**2))
                print("Error in pixel: ", mean_error_pix[-1])
                print("Error angle: ", mean_error_angle[-1])
            print(key, "Coping skymap and blob ", blobs_key, " to 'dataset_none' folder.")
            origin = src_folder+"/skymap_NONE_"+str(key)+".fits"
            dest = "data/dataset_none/skymap_NONE_"+str(key)+"_"+str(SOURCES_NUMBER)+".fits"
            #shutil.copyfile(origin, dest)
            # Add contents of list as last row in the csv file
            #csv_writer.writerow(to_be_saved)
print("Mean error pixel: ", np.mean(mean_error_pix))
print("Max error in pixel: ", np.max(mean_error_pix))
print("Min error in pixel: ", np.min(mean_error_pix))
print("Standard deviation error pixel: ", np.std(mean_error_pix))
print("Mean error angle: ", np.mean(mean_error_angle))
print("Max error in angle: ", np.max(mean_error_angle))
print("Min error in angle: ", np.min(mean_error_angle))
print("Standard deviation error angle: ", np.std(mean_error_angle))